# Redes Neuronales

Se realizó el entrenamiento del modelo con los datos del set de fashion-mnist, en el cual se tienen imágenes de 10 tipos distintos de prendas de ropa, representadas numéricamente en 784(28x28) pixeles. Para el entrenamiento se tomaron 60,000 datos, traducidos a una sola matriz de 60,000x784. En el repositorio de este notebook se incluye neural_networks.py, ahí se realizó el entrenamiento del modelo con el training set, el cual se guardó en el archivo 'data/training_model'. Debido al largo tiempo que se tomó en finalizar el entrenamiento, no se repetirá en este notebook pero el código sí quedará aquí. 

## Definición de funciones 

A continuación se definirán las funciones necesarias para la implementación de la red neuronal. Estas se encuentran archivadas en el módulo de 'back_propagation.py' (en donde también se encuentra una mayor explicación de los parámetros, los valores devueltos y comentarios en las secciones relevantes), por lo que solo se describirá aquí las funciones más relevantes y no se correrán aquí ya que se realiza al importar dicho módulo. 

### Sigmoide

Esta función está descrita por el siguiente modelo matemático: $$ S(x) = \frac{1}{1 + e^{-z}}$$


In [ ]:
def sigmoid(z):
    return 1.0/(1 + np.exp(-z))

### Feed forward

In [ ]:
def feed_forward(thetas, X):
    a = [X] 
    for i in range(len(thetas)): 
        a.append(
            sigmoid(
                np.matmul(
                    np.hstack((
                        np.ones(len(a[i])).reshape(len(a[i]), 1),
                        a[i]
                    )),
                    thetas[i].T
                )
            )
        )
    return a 

### Back propagation

In [ ]:
def back_propagation(f_thetas, shapes, X, Y):
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(f_thetas, shapes)
    a = feed_forward(thetas, X)
    deltas = [ *range(layers -1), a[-1] - Y ] 
    for i in range(layers-2, 0, -1):
        deltas[i] = np.matmul(
            deltas[i+1],
            (thetas[i])[:, 1:]
        ) * (a[i] * (1 - a[i]))
    gradient = []
    for i in range(layers-1):
        gradient.append((np.matmul(
            deltas[i+1].T, 
            np.hstack(( 
                        np.ones(len(a[i])).reshape(len(a[i]), 1),
                        a[i]
                    ))
        )) / m)
    return flatten_matrixes(gradient)

### Función de costo 

La función de costo se realiza haciendo la sumatoria de diferencias entre el valor esperado y el valor proveído por cada neurona, en cada capa. El modelo matemático es el siguiente: 
$$ J(\theta) =  \frac{-1}{m}\sum_{i=1}^{m} \sum_{k=1}^{K} y_k^i log(h_\theta (x^i)_k)  +  (1-y_k^i)log(1 - h_\theta (x^i)_k)$$
En donde:

K: cantidad de capas 

m: cantidad de datos ingresados

In [ ]:
def nn_cost(flat_thetas, shapes, X, Y):
    # obtener predicciones 
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)

In [1]:
#import de todas las librerías necesarias 
from back_propagation import * # en este modulo se tienen las funciones para la red neuronal 
import numpy as np
import mnist_reader
import scipy.optimize as optimize
import pickle

## Training

El siguiente código fue realizado para la parte de training del modelo. 

In [2]:
# data import 
X_train, y_train = mnist_reader.load_mnist('data/', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/', kind='t10k')

In [3]:
X = X_train / 255.0
m,n = X.shape

In [4]:
# establecimiento de constantes
HIDDEN_LAYER = 90 # Cantidad de neuronas en la capa oculta
FINAL_LAYER = 10 # cantidad de neuronas en la capa final 
theta_shapes = np.array([
    [ HIDDEN_LAYER, n+1 ],
    [ FINAL_LAYER, HIDDEN_LAYER+1 ]
])
theta_shapes_list = [(HIDDEN_LAYER, n+1), (FINAL_LAYER, HIDDEN_LAYER+1)]

In [5]:
# la función de costo y back_propagation requiere comparación de matrices de predicción y valores teóricos (Y), por lo
# que se debe traducir el vector de datos teóricos en una matriz 60,000x10
Y = np.zeros((X.shape[0], FINAL_LAYER))
for i in range(m):
    Y[i][y_train[i]] = 1

El siguiente código se presentará solo para demostrar cómo se realizó el entrenamiento, pero no se ejecutará aquí. 

In [ ]:
# generacion de matrices de transicion (aplanadas) random 
flat_thetas = flatten_matrixes([
    np.random.rand(*theta_shape) / 255.0
    for theta_shape in theta_shapes
])

In [ ]:
# optimización de modelo 
result = optimize.minimize(
    fun=nn_cost,
    x0=flat_thetas,
    args=(theta_shapes, X, Y),
    method='L-BFGS-B',
    jac=back_propagation,
    options={
        'disp': True, 
        'maxiter': 1000, 
    }
)

In [ ]:
# upload de modelo a un archivo
model_filename = 'data/trained_model'
model_file = open(model_filename, 'wb')
# writing model 
pickle.dump(np.asarray(result.x), model_file)
model_file.close()

Ahora, se hará el load del modelo optimizado por el código anterior y se obtendrá su porcentaje de accuracy para los datos de training. El accuracy se obtuvo de la siguiente manera: $$ accuracy = \frac{\# hits}{\# instances}$$

In [14]:
model_filename = 'data/trained_model'

In [7]:
model_file = open(model_filename, 'rb')
flat_thetas = pickle.load(open(model_filename, 'rb'))
model_file.close()

In [8]:
#inflar las matrices flattened que vienen del modelo cargado 
result_thetas = inflate_matrixes(flat_thetas, theta_shapes_list)

In [9]:
#obtener la matriz de prediccion (ultima que da el feed forward)
prediction = feed_forward(result_thetas, X)[-1]

In [10]:
accuracy = get_accuracy(prediction, y_train)
print("TRAINING ACCURACY: {}%".format(accuracy * 100))

TRAINING ACCURACY: 98.85666666666667%


También se obtendrá el porcentaje de accuracy por clase. Las clases son como sigue:

0:	T-shirt/top

1:	Trouser

2:	Pullover

3:	Dress

4:	Coat

5:	Sandal

6:	Shirt

7:	Sneaker

8:	Bag

9:	Ankle boot

In [11]:
accuracy_per_class = get_accuracy_by_class(prediction, y_train)

In [13]:
for value in accuracy_per_class:
    print('Clase {}: {:.2f}%'.format(accuracy_per_class.index(value), value * 100))

Clase 0: 98.60%
Clase 1: 99.88%
Clase 2: 96.78%
Clase 3: 98.53%
Clase 4: 98.13%
Clase 5: 100.00%
Clase 6: 97.13%
Clase 7: 99.98%
Clase 8: 99.58%
Clase 9: 99.93%


## Testing

Ahora se realizará la predicción para los datos de testing. Para este set se tomaron 10,000 del archivo 't10k' proveído por el creador de fashion-mnist. 

In [15]:
#obtener la matriz de prediccion (ultima que da el feed forward)
X = X_test / 255.0
m,n = X.shape

In [16]:
theta_shapes = np.array([
    [ HIDDEN_LAYER, n+1 ],
    [ FINAL_LAYER, HIDDEN_LAYER+1 ]
])
theta_shapes_list = [(HIDDEN_LAYER, n+1), (FINAL_LAYER, HIDDEN_LAYER+1)]

In [17]:
result_thetas = inflate_matrixes(flat_thetas, theta_shapes_list)

In [18]:
prediction = feed_forward(result_thetas, X)[-1]

In [19]:
test_accuracy = get_accuracy(prediction, y_test)
print("TEST ACCURACY: {}%".format(test_accuracy * 100))

TEST ACCURACY: 85.5%


In [20]:
test_accuracy_per_class = get_accuracy_by_class(prediction, y_test)

In [21]:
for value in test_accuracy_per_class:
    print('Clase {}: {:.2f}%'.format(test_accuracy_per_class.index(value), value * 100))

Clase 0: 79.50%
Clase 1: 96.60%
Clase 2: 75.30%
Clase 3: 84.10%
Clase 4: 77.40%
Clase 5: 94.90%
Clase 6: 62.70%
Clase 7: 94.40%
Clase 8: 95.30%
Clase 9: 94.80%


## Conclusiones

En su mayoría, el modelo tiene un buen desempeño, con un 85.5% de aciertos en general. Los datos que más le cuesta trabajo predecir y clasificar correctamente son aquellos que pertenecen a las clases 6, 4, 2, y 0, que son las camisas, los abrigos, los pullovers y t-shirts, respectivamente. La razón por la cual se confunde tanto puede ser debido a que estas prendas en específico tienden a parecerse (a veces hasta uno de humano tiene dificultades en diferenciar abrigos y suéteres y pullovers o camisas - shirts - y t-shirts), por lo que es comprensible que el algoritmo se confunda y clasifique un objeto en otra clase parecida. Lo importante es que sabe diferenciar bastante bien entre objetos que son fácilmente separables, como bolsas, sandalias, sneakers y botitas. 